# MPI Introduction

An introduction to basic concept of MPI

## MPI Init and Finalize

Calls used to initialize and terminate the parallel session.

* `MPI_Init`: see https://www.open-mpi.org/doc/v3.1/man3/MPI_Init.3.php
* `MPI_Finalize`: see https://www.open-mpi.org/doc/v3.1/man3/MPI_Finalize.3.php

The following program initialize the parallel session.

**NB**: The arguments in `MPI_Init` are **not used** anymore but some compilers insist they are there.

In [ ]:
%%capture
%%writefile main_init.cpp

#include <iostream>
#include <mpi.h>

int main(int argc, char **argv) 
{
    // Initialize MPI
    // This must always be called before any other MPI functions
    MPI_Init(&argc, &argv);
    
    std::cout<< "Hello world!"<< std::endl;

    // Finalize MPI
    // This must always be called after all other MPI functions
    MPI_Finalize();

    return 0;
}

In [ ]:
%%sh

# compile program
mkdir -p ./Debug
cd Debug
cmake -DSOURCES="main_init.cpp" ..
make

In [39]:
%%sh

# run program
cd Debug
mpirun -np 4 program

Hello I am process 0 of 4 processes
Hello I am process 1 of 4 processes
Hello I am process 2 of 4 processes
Hello I am process 3 of 4 processes


## MPI Communicators

It is possible to divide the total number of tasks into groups called **communicators**.
The variable identifying a communicator identifies those tasks which can communicate with each other.

**NB** The default communicator is called `MPI_COMM_WORLD` and by default includes all the tasks available to the program.

<img src="./Images/COMM_WORLD.png" width=20% style="margin-left:auto; margin-right:auto">


* `MPI_Comm_size`: see https://www.open-mpi.org/doc/v3.1/man3/MPI_Comm_size.3.php
* `MPI_Comm_rank`: see https://www.open-mpi.org/doc/v3.1/man3/MPI_Comm_rank.3.php

In [36]:
%%capture
%%writefile main_communicators.cpp

#include <iostream>
#include <mpi.h>

int main(int argc, char **argv)
{
    int err;
    err = MPI_Init(&argc, &argv);
    
    int nprocs, my_rank;
    
    // Get the number of processes in MPI_COMM_WORLD
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);
    
    // Get the rank of this process in MPI_COMM_WORLD
    MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);

    std::cout<< "Hello I am process "<< my_rank<< " of "<< nprocs<< " processes"<< std::endl; 
    
    err = MPI_Finalize();
    
    return 0;
}

In [37]:
%%sh

# compile program
mkdir -p ./Debug
cd Debug
cmake -DSOURCES="main_communicators.cpp" ..
make

-- Configuring done
-- Generating done
-- Build files have been written to: /home/geoscore/Desktop/GEO++/Courses/PhdMPI/1_MPI_Basic/Debug
Consolidate compiler generated dependencies of target program
[ 50%] Building CXX object CMakeFiles/program.dir/main_communicators.cpp.o
[100%] Linking CXX executable program
[100%] Built target program


In [40]:
%%sh

# run program
cd Debug
mpirun -np 4 program

Hello I am process 0 of 4 processes
Hello I am process 1 of 4 processes
Hello I am process 2 of 4 processes
Hello I am process 3 of 4 processes
